# 🧠 NeuroTrader Ultimate: Cloud Training (Colab)

Welcome to the high-performance training environment. This notebook allows you to train the NeuroTrader AI agent using Google Colab's powerful resources.

### Steps:
1. **Setup**: Install dependencies.
2. **Connect Drive**: Mount Google Drive to save logs/models persistently.
3. **Upload Data**: Upload your `.parquet` processed data.
4. **Define Environment**: Initialize the `TradingEnv`.
5. **Train**: Run the PPO algorithm for millions of timesteps.
6. **Download**: Save the trained brain (`model.zip`) to use locally.

## 1. Setup Environment
**Note:** If you see errors about `numpy` or `gym`, try clicking **Runtime > Restart Session** and running this cell again.

In [ ]:
# Install dependencies with version pinning to avoid Numpy 2.0 conflicts
# --force-reinstall ensures we get binaries compiled against the OLD numpy, not the pre-installed new one
!pip install "numpy<2.0" gymnasium==0.29.1 "stable-baselines3[extra]>=2.3.0" pandas ta pyarrow shimmy>=1.3.0 --force-reinstall

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvaluatorCallback, CheckpointCallback
from stable_baselines3.common.monitor import Monitor
from typing import Optional
import os
import glob
from google.colab import drive

# Check GPU availability
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using Device: {device}")

## 2. Connect Google Drive ☁️
Connect to Drive to save logs and models automatically.

In [ ]:
drive.mount('/content/drive')

# Defines where to save logs
# Usually maps to: /content/drive/MyDrive/...
# You can change this path to match your folder ID if needed, but standard path is safest.
LOG_DIR = "/content/drive/MyDrive/NeuroTrader_Workspace/logs"
MODEL_DIR = "/content/drive/MyDrive/NeuroTrader_Workspace/models"

os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
print(f"✅ Logging to: {LOG_DIR}")
print(f"✅ Models will save to: {MODEL_DIR}")

## 3. Upload Data
Please upload your processed Parquet files (e.g. `BTCUSD_M1.parquet`) to the Colab runtime (drag and drop to the file panel on the left).

In [ ]:
# Load Data (Dynamic Detection)
parquet_files = glob.glob("*.parquet")

if parquet_files:
    print(f"📂 Found {len(parquet_files)} data files: {parquet_files}")
    
    # Select the first one by default, or change index to select others
    DATA_FILE = parquet_files[0]
    print(f"👉 Training Target: {DATA_FILE}")
    
    df = pd.read_parquet(DATA_FILE)
    print(f"✅ Loaded Data: {len(df):,} rows")
    print(df.head())
else:
    print("❌ No .parquet files found! Please upload your data to the Files panel on the left.")

## 4. Define Trading Environment
(Self-contained class to avoid file dependencies)

In [ ]:
class TradingEnv(gym.Env):
    metadata = {'render_modes': ['human']}

    def __init__(self, df: pd.DataFrame, initial_balance=10000, max_steps=None):
        super(TradingEnv, self).__init__()
        self.df = df
        self.initial_balance = initial_balance
        self.max_steps = max_steps if max_steps else len(df) - 1
        
        # Actions: 0=HOLD, 1=BUY, 2=SELL
        self.action_space = spaces.Discrete(3)
        
        # Features expected in DF
        self.feature_cols = [
            'close', 'rsi', 'macd', 'macd_signal', 
            'bb_high', 'bb_low', 'ema_20', 'ema_50'
        ]
        
        # Check cols
        missing = [c for c in self.feature_cols if c not in df.columns]
        if missing: raise ValueError(f"Missing cols: {missing}")

        # Obs: Features + Balance + Position
        # Using Float32 for compatibility with PPO MlpPolicy
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(len(self.feature_cols) + 2,), dtype=np.float32
        )
        
        self.current_step = 0
        self.balance = initial_balance
        self.position = 0.0
        self.equity = initial_balance

    def reset(self, seed: Optional[int] = None, options: Optional[dict] = None):
        super().reset(seed=seed)
        self.current_step = 0
        self.balance = self.initial_balance
        self.position = 0.0
        self.equity = self.initial_balance
        return self._get_observation(), {}

    def _get_observation(self):
        row = self.df.iloc[self.current_step]
        obs = [row[col] for col in self.feature_cols]
        obs.append(self.balance)
        obs.append(self.position)
        return np.array(obs, dtype=np.float32)

    def step(self, action):
        current_price = self.df.iloc[self.current_step]['close']
        prev_equity = self.equity
        
        # Logic: Fixed fractional or simple all-in for logic training
        # Let's simple logical units: Buy = spend available / Sell = sell all
        fee_rate = 0.001
        
        if action == 1 and self.balance > 0: # BUY
            cost = self.balance
            fee = cost * fee_rate
            units = (cost - fee) / current_price
            self.position += units
            self.balance = 0
            
        elif action == 2 and self.position > 0: # SELL
            revenue = self.position * current_price
            fee = revenue * fee_rate
            self.balance += (revenue - fee)
            self.position = 0

        self.current_step += 1
        
        # Equity calc
        crypto_val = self.position * self.df.iloc[self.current_step]['close'] if self.current_step < len(self.df) else 0
        self.equity = self.balance + crypto_val
        
        # Reward: Log Return
        reward = 0
        if prev_equity > 0:
            reward = np.log(self.equity / prev_equity) * 100 # Scaling for stability
            
        done = self.current_step >= len(self.df) - 1
        if self.equity < self.initial_balance * 0.1: # Bust (<10%)
            done = True
            reward = -100
            
        return self._get_observation(), reward, done, False, {'equity': self.equity}

## 5. Train Agent 🏋️
We use **PPO (Proximal Policy Optimization)**. It's stable and effective.

In [ ]:
# Create Env
env = TradingEnv(df)
env = Monitor(env) # For logging

# Callbacks
checkpoint_callback = CheckpointCallback(
    save_freq=100000, # Save model every 100k steps
    save_path=MODEL_DIR,
    name_prefix="neurotrader_ppo"
)

# Define Model
model = PPO(
    "MlpPolicy", 
    env, 
    verbose=1, 
    device=device,
    tensorboard_log=LOG_DIR, # Save logs to Drive
    learning_rate=0.0003,
    n_steps=2048,
    batch_size=64,
    gamma=0.99
)

# Train
print("🚀 Training Started... (Logs are saved to your Drive)")
model.learn(
    total_timesteps=3_000_000, # Increased for serious training
    progress_bar=True,
    callback=checkpoint_callback
)
print("✨ Training Complete!")

## 6. Save Final Model 💾
The model is already saved in Drive via checkpoints, but we force a final save here.

In [ ]:
final_path = os.path.join(MODEL_DIR, "neurotrader_brain_final")
model.save(final_path)
print(f"Final Model saved to: {final_path}.zip")

## 7. Visualize (TensorBoard)
Run this cell to see training graphs inside Colab.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "$LOG_DIR"